In [12]:
# 新闻爬取xlxs文件目录：
NEWS_PATH = '../news/'
#语料库文件路径：
DATA_PATH = '../data/news_words.txt'
#标记文件路径
LABEL_PATH = '../data/news_labels.txt'
#标记数据路径
LABEL_DIR = '../labels/'
# LDA Model 路径：
LDA_PATH = '../model/lda.model'
# Dictionary 路径
DICT_PATH = '../model/dictionary.txtdic'
# Random Forest 模型路径
RF_PATH = '../model/rf.model'
#coding=utf-8  
import codecs  
from gensim import corpora  
from gensim.models import LdaModel  
from gensim.corpora import Dictionary  


import jieba
import csv
import numpy as np  
import os  
import pandas as pd
from sklearn.naive_bayes import GaussianNB  
from sklearn.externals import joblib

import time    
from sklearn import metrics    
import pickle as pickle    
import pandas as pd  
  
import random


import shutil
import time
import word_segmentation as ws
word_segmentor = ws.WordSegmentation()

# 由CSV生成语料库

* 读取csv文件
* 对于一行资讯进行分词
* 将分词结果写入新的txt文件

In [1]:
# 常量定义

#语料库文件路径：
DATA_PATH = '../data/news_words.txt'

import shutil
import time
import pandas as pd
import word_segmentation as ws
word_segmentor = ws.WordSegmentation()


# write_news_words
# 输入 csv 文件的 file_path
# 操作：分词-写入DATA_FILE
def write_news_words(file_path):
    file_data = pd.read_csv(file_path)
    file_data.rename(columns={'标题':'title', '正文':'content','正文1':'content',"字段1_文本":"title"}, inplace = True)
    file = open(DATA_PATH,"a")
    try:
        for index, row in file_data.iterrows():
            #print(row.content)
            row_words = word_segmentor.word_segmentation(str(row.title) + str(row.content))
            file_words = " ".join(row_words)
            file.write(file_words + ' \n')
    except:
        print("Error reading news from the file:", file)
    file.close()
    

# 添加文件到语料库
# 输入：new_path 为新增的抓取csv数据的文件夹
# 执行：将csv文件的分词结果写入语料库文件
def append_csv2txt(new_path,old_path):
    files = os.listdir(new_path)
    print(files)
    
    for fname in files:
        fpath = new_path + fname
        if 'csv' in fpath:
            print(fpath)
            write_news_words(fpath)
            shutil.move(fpath,old_path)
            

In [2]:
# 将 news 文件夹中的csv文件读取存入语料库，并将csv文件移至 corpus 文件夹中

start_time = time.time()
append_csv2txt('../news/','../corpus/')
print('Loading CSV files took %fs!' % (time.time() - start_time)) 


NameError: name 'os' is not defined

# 标记数据集生成

对标记数据csv进行读取-分词 的转换，并将结果连同label一起写入txt文件


In [2]:

# write_news_words
# 输入 csv 文件的 file_path
# 操作：分词-写入DATA_FILE
def write_news_labels(fpath):
    print("Reading label csv file:",fpath)
    file_data = pd.read_csv(fpath)
    file_data.rename(columns={'标题':'title', '正文':'content','正文1':'content',"字段1_文本":"title","关键词":"plate"}, inplace = True)
    file = open(LABEL_PATH,"a")
    try:
        for index, row in file_data.iterrows():
            #print(row.content)
            #news_word = jieba_split(str(row.title) + str(row.content))
            news_word = " ".join(word_segmentor.word_segmentation(str(row.title) + str(row.content)))
            #news_bow = dictionary.doc2bow(news_word)      #文档转换成bow  
            #news_lda = lda2list(lda[news_bow],301) #得到lda向量
            file.write(row.plate + "," + news_word + "\n")
            #print(row.plate,news_lda)
    except:
        print("Error reading labels from the file:", fpath)
    file.close()
    

# 添加文件到语料库
# 输入：new_path 为新增的抓取csv数据的文件夹
# 执行：将csv文件的分词结果写入语料库文件
def append_labels2txt(new_path,old_path):
    files = os.listdir(new_path)
    print(files)
    for fname in files:
        fpath = new_path + fname
        if 'csv' in fpath:
            print(fpath)
            write_news_labels(fpath)
            shutil.move(fpath,old_path)
            

In [3]:

start_time = time.time()
append_labels2txt('../labels/','../label_old/')
print('Loading CSV files took %fs!' % (time.time() - start_time)) 


['东财关键词对应新闻-0426-6.csv', '东方财经关键词板块对应新闻-0425-2.csv', '东方财经关键词板块对应新闻-0425-1.csv', '东财关键词对应新闻-0427.csv', '行业研究报告数据中心东方财富网-0502-2.csv', '行业研究报告数据中心东方财富网-0502-1.csv', '东财关键词对应新闻-0426-7.csv', '东财关键词对应新闻-0426-4.csv', '东财关键词对应新闻-0426-3.csv', '东财关键词对应新闻-0426-8.csv', '东方财经关键词板块对应新闻-0427.csv', '东财关键词对应新闻-0426-1.csv', '东财关键词对应新闻-0426-5.csv', '东财关键词对应新闻-0426-2.csv']
../labels/东财关键词对应新闻-0426-6.csv
Reading label csv file: ../labels/东财关键词对应新闻-0426-6.csv


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.890 seconds.
Prefix dict has been built succesfully.


Error reading labels from the file: ../labels/东财关键词对应新闻-0426-6.csv


Error: Destination path '../label_old/东财关键词对应新闻-0426-6.csv' already exists

# 离线语料分析建模及板块预测模型优化

LDA：

* 生成语料库（初期由csv文件生成，后期由数据集中获取）
* 读取语料库（每行一条资讯，已分词，空格分隔）
* 利用语料库训练LDA模型并保存

RF：

* 读取标记数据库（每行一条资讯，第一列为标签，第二列为资讯，已分词，空格分隔）
* 利用LDA模型转换资讯为Vector-标记 格式
* 训练RF模型并保存

## 语料库载入及LDA模型训练

In [7]:
class MyCorpus(object):
    def __iter__(self):
        for line in open(DATA_PATH):
            # assume there's one document per line, tokens separated by whitespace
            yield dictionary.doc2bow(line.split(' '))

In [14]:
# 遍历语料库文件，逐步增加dictionary
print("Start reading corpus file...")
start_time = time.time()    
fr=open(DATA_PATH,'r')  
train=[]  
corpus_mem_friendly = MyCorpus()
# dictionary = corpora.Dictionary(corpus_mem_friendly)
dictionary = corpora.Dictionary(line.split(' ') for line in open(DATA_PATH))
print('Composing dictionary took %fs!' % (time.time() - start_time)) 
start_time = time.time()
lda = LdaModel(corpus=corpus_mem_friendly, id2word=dictionary, num_topics=400)  
lda.save(LDA_PATH)
print('LDA training took %fs!' % (time.time() - start_time)) 


Start reading corpus file...
Composing dictionary took 269.134172s!


/home/inesa/.local/lib/python3.5/site-packages/gensim/models/ldamodel.py:775: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


LDA training took 11300.780501s!


In [13]:
dictionary.save(DICT_PATH)

In [16]:
# 打印LDA模型相关结果
topic_list=lda.print_topics(40)  

for topic in topic_list:  
    print(topic) 
print("第一主题")
print(lda.print_topic(1))

(124, '0.390*"银行" + 0.055*"中国" + 0.039*"商银" + 0.031*"民生" + 0.026*"分行" + 0.024*"生银" + 0.024*"工商" + 0.024*"平安" + 0.022*"民生银行" + 0.021*"中国银行"')
(256, '0.039*"市场" + 0.023*"投资" + 0.018*"估值" + 0.018*"A股" + 0.017*"成长" + 0.016*"行情" + 0.016*"机会" + 0.012*"蓝筹" + 0.012*"股" + 0.011*"资金"')
(251, '0.079*"持股" + 0.064*"流通" + 0.049*"社保" + 0.047*"十大" + 0.045*"个股" + 0.045*"万股" + 0.035*"比例" + 0.033*"持有" + 0.029*"流通股" + 0.026*"新进"')
(230, '0.234*"美元" + 0.163*"亿美元" + 0.041*"美国" + 0.030*"公司" + 0.024*"万美元" + 0.024*"年" + 0.018*"全球" + 0.017*"亚马" + 0.017*"亚马逊" + 0.015*"超过"')
(175, '0.040*"成本" + 0.036*"税收" + 0.033*"降低" + 0.030*"政策" + 0.028*"所得" + 0.027*"增值" + 0.027*"企业" + 0.025*"增值税" + 0.024*"收费" + 0.023*"所得税"')
(203, '0.094*"工厂" + 0.078*"自动" + 0.068*"自动化" + 0.058*"生产" + 0.043*"模具" + 0.042*"工业" + 0.038*"供应" + 0.034*"制造" + 0.028*"供应商" + 0.028*"订单"')
(320, '0.089*"汽车" + 0.021*"车型" + 0.020*"上汽" + 0.019*"大众" + 0.019*"汽车产业" + 0.019*"自主" + 0.018*"销量" + 0.018*"品牌" + 0.017*"一汽" + 0.015*"长安"')
(38, '0.143*"证监" + 0.127*"证监会

# 标签库载入及分类器训练

* 标记数据来源： 第一财经新闻等按关键词搜索的新闻爬取结果
* 标记格式：新闻标题/正文/关键词

训练过程：

* 读取含有“关键词”-“标题”-“正文” 的csv文件
* 对文件中的 标题 + 正文 字符串进行分词
* 分词结果使用LDA模型向量化
* 向量 ： 关键词 作为 Random Forest模型的训练输入



In [4]:
start_time = time.time()
lda = LdaModel.load(LDA_PATH)
print('Loading LDA model took %fs!' % (time.time() - start_time)) 

Loading LDA model took 2.170682s!


In [8]:
import word_segmentation
# 对于label文件夹下的CSV，读取后将标题+正文转换为分词结果，再转换为LDA结果，形成200维向量
# 对于新闻的标签，直接保留其表格中的“关键词”地段

corpus_mem_friendly = MyCorpus()
# dictionary = corpora.Dictionary(corpus_mem_friendly)
dictionary = corpora.Dictionary(line.split(' ') for line in open(DATA_PATH))

#df_label = pd.DataFrame(columns=['words','label'])  
news_vec = []
news_labels = []

# turn lda result into list
def lda2list(lda,topic_n):
    lda_dict = dict(lda)
    lda_list = [0] * topic_n
    for i in range(topic_n):
        lda_list[i] = lda_dict.get(i,0)
    return lda_list

start_time = time.time()
print("Start loading labeled data...")
files = os.listdir(LABEL_DIR)
print(files)
for fname in files:
    fpath = LABEL_DIR + fname
    if 'csv' in fpath:
        print(fpath)
        file_data = pd.read_csv(fpath)
        file_data.rename(columns={'标题':'title', '正文':'content','正文1':'content',"字段1_文本":"title","关键词":"plate"}, inplace = True)
        for index, row in file_data.iterrows():
            #print(row.content)
            #news_word = jieba_split(str(row.title) + str(row.content))
            news_word = word_segmentor.word_segmentation(str(row.title) + str(row.content))
            news_bow = dictionary.doc2bow(news_word)      #文档转换成bow  
            news_lda = lda2list(lda[news_bow],301) #得到lda向量
            news_vec.append(news_lda)
            news_labels.append(row.plate)
            #print(row.plate,news_lda)
    print("done")
print('Loading labeled data took %fs!' % (time.time() - start_time)) 
        

Start loading labeled data...
['东财关键词对应新闻-0426-6.csv', '东方财经关键词板块对应新闻-0425-2.csv', '东方财经关键词板块对应新闻-0425-1.csv', '东财关键词对应新闻-0427.csv', '行业研究报告数据中心东方财富网-0502-2.csv', '行业研究报告数据中心东方财富网-0502-1.csv', '东财关键词对应新闻-0426-7.csv', '东财关键词对应新闻-0426-4.csv', '东财关键词对应新闻-0426-3.csv', '东财关键词对应新闻-0426-8.csv', '东方财经关键词板块对应新闻-0427.csv', '东财关键词对应新闻-0426-1.csv', '东财关键词对应新闻-0426-5.csv', '东财关键词对应新闻-0426-2.csv']
../labels/东财关键词对应新闻-0426-6.csv
done
../labels/东方财经关键词板块对应新闻-0425-2.csv
done
../labels/东方财经关键词板块对应新闻-0425-1.csv
done
../labels/东财关键词对应新闻-0427.csv
done
../labels/行业研究报告数据中心东方财富网-0502-2.csv
done
../labels/行业研究报告数据中心东方财富网-0502-1.csv
done
../labels/东财关键词对应新闻-0426-7.csv
done
../labels/东财关键词对应新闻-0426-4.csv
done
../labels/东财关键词对应新闻-0426-3.csv
done
../labels/东财关键词对应新闻-0426-8.csv
done
../labels/东方财经关键词板块对应新闻-0427.csv
done
../labels/东财关键词对应新闻-0426-1.csv
done
../labels/东财关键词对应新闻-0426-5.csv
done
../labels/东财关键词对应新闻-0426-2.csv
done
Loading labeled data took 43836.135314s!


In [9]:
   
# Random Forest Classifier    
def random_forest_classifier(train_x, train_y):    
    from sklearn.ensemble import RandomForestClassifier    
    model = RandomForestClassifier(n_estimators=8)    
    model.fit(train_x, train_y)    
    return model    
    

X = np.array([np.array(xi) for xi in news_vec])
Y = np.array(news_labels)

print("********classifier: random forest ***********")
start_time = time.time()    
model = random_forest_classifier(X, Y)
print('training took %fs!' % (time.time() - start_time)) 
for i in range(100):
    n = random.randrange(1,len(X))
    # predict_proba(x)：给出带有概率值的结果。每个点在所有label的概率和为1.  
    # predict(x)：直接给出预测结果。内部还是调用的predict_proba()，根据概率的结果看哪个类型的预测值最高就是哪个类型。  
    print(model.predict([X[n]]))
    print(Y[n])

********classifier: random forest ***********
training took 40.354848s!
['文化传媒']
材料行业
['珠宝首饰']
珠宝首饰
['高速公路']
高速公路
['电子信息']
电子信息
['公用事业']
公用事业
['公用事业']
公用事业
['保险']
银行
['保险']
保险
['贵金属']
贵金属
['房地产']
房地产
['贵金属']
贵金属
['电子信息']
电子信息
['高速公路']
高速公路
['石油行业']
石油行业
['港口水运']
船舶制造
['银行']
银行
['家电行业']
家电行业
['医疗行业']
医疗行业
['高速公路']
高速公路
['汽车行业']
汽车行业
['保险']
保险
['文教休闲']
文教休闲
['文化传媒']
文化传媒
['家电行业']
汽车行业
['汽车行业']
汽车行业
['电子信息']
高速公路
['银行']
银行
['工艺商品']
工艺商品
['农药兽药']
文教休闲
['电子信息']
电子信息
['公用事业']
公用事业
['纺织服装']
纺织服装
['珠宝首饰']
珠宝首饰
['贵金属']
贵金属
['汽车行业']
汽车行业
['材料行业']
材料行业
['高速公路']
高速公路
['贵金属']
贵金属
['化工行业']
石油行业
['纺织服装']
纺织服装
['文教休闲']
文教休闲
['贵金属']
贵金属
['房地产']
房地产
['仪器仪表']
仪器仪表
['汽车行业']
汽车行业
['房地产']
房地产
['电子信息']
电子信息
['有色金属']
有色金属
['化工行业']
化工行业
['汽车行业']
汽车行业
['高速公路']
高速公路
['汽车行业']
汽车行业
['电信运营']
电信运营
['文化传媒']
文化传媒
['化工行业']
化工行业
['贵金属']
贵金属
['机械行业']
机械行业
['纺织服装']
纺织服装
['家电行业']
家电行业
['纺织服装']
纺织服装
['化工行业']
化工行业
['煤炭采选']
煤炭采选
['交运物流']
交运物流
['公用事业']
公用事业
['高速公路']
高速公路
['房地产']
软件服务
['文化传媒']
文化传媒
['有色金属']
有色金属
['家电行业']
家电行业
[

In [14]:
for i in range(100):
    n = random.randrange(1,len(X))
    # predict_proba(x)：给出带有概率值的结果。每个点在所有label的概率和为1.  
    # predict(x)：直接给出预测结果。内部还是调用的predict_proba()，根据概率的结果看哪个类型的预测值最高就是哪个类型。  
    print([X[n]])
    print(model.predict([X[n]]))
    print(Y[n])

[array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.04958371, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.    

['园林工程']
园林工程
[array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.07968206, 0.        , 0.        , 0.        ,
       0.01286432, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.01571349,
       0.04910782, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.  

['港口水运']
港口水运
[array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.07983214, 0.03590532, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.01087864, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.02667765, 0.        , 0.0103878 , 0.01586777,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.0296883 , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.01271323, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.  

In [11]:
joblib.dump(model, RF_PATH)

['../model/rf.model']